In [138]:
import os
from resolve_api_samples.Examples.python_get_resolve import GetResolve
from pprint import pprint
import sys
from typing import List
from collections import Counter

resolve = GetResolve()
project_manager = resolve.GetProjectManager()
project = project_manager.GetCurrentProject()
media_storage = resolve.GetMediaStorage()
media_pool = project.GetMediaPool()
root_folder = media_pool.GetRootFolder()

media_path = '/Volumes/thom_7_for_Mac/tmp/素材'
sub_folders_full_path = media_storage.GetSubFolderList(media_path)
INVALID_EXTENSION = ["DS_Store", "JPG", "JPEG", "SRT"]  # TODO, 小写的情况还待考虑进去


def absolute_file_paths(directory) -> list:
    absolute_file_path_list = []
    for directory_path, _, filenames in os.walk(directory):
        for f in filenames:
            absolute_file_path_list.append(os.path.abspath(os.path.join(directory_path, f)))
    return absolute_file_path_list


def get_sub_folder_by_name(sub_folder_name: str):
    """Get folder object by name."""
    all_sub_folder = root_folder.GetSubFolderList()
    sub_folder_dict = {sub_folder.GetName(): sub_folder for sub_folder in all_sub_folder}
    return sub_folder_dict.get(sub_folder_name, "")


# def import_clip() -> None:
#     """Import footage from media storage into the corresponding sub-folder of the media pool root folder."""
#     for count, sub_folder in enumerate(root_folder.GetSubFolderList()):
#         media_pool.SetCurrentFolder(sub_folder)
#         # 导入素材的时候排除 _Timeline 这个目的地 Bin
#         if sub_folder.GetName() == "_Timeline":
#             break
#         media_storage.AddItemListToMediaPool(sub_folders_full_path[count])


def get_sub_folder_name(source_media_full_path: List[str]) -> List[str]:
    """
    Extract sub-folder name from media storage full path.
    For creating sub-folder in the media pool.
    """
    sub_folders_name = []
    for i in source_media_full_path:
        split_full_path = i.split("/")
        sub_folders_name.append(split_full_path[-1])
    return sub_folders_name


# def import_clip_new() -> None:
media_full_path_list = absolute_file_paths(media_path)
filename_and_fullpath_dict = {os.path.splitext(path)[0].replace(".", "").split('/')[-1]: path for path in
                              media_full_path_list if path.split(".")[-1] not in INVALID_EXTENSION}
filename_and_fullpath_keys = list(filename_and_fullpath_dict.keys())
filename_and_fullpath_keys.sort()
filename_and_fullpath_value = [filename_and_fullpath_dict.get(i) for i in filename_and_fullpath_keys]

camname_and_fullpath_dict = {path: path.split('/')[path.split('/').index('素材') + 1] for path in filename_and_fullpath_value
                             if path.split('.')[-1] not in INVALID_EXTENSION}

cam_list = [path.split('/')[path.split('/').index('素材') + 1] for path in filename_and_fullpath_value]
counts = Counter(cam for cam in cam_list)

{'/Volumes/thom_7_for_Mac/tmp/素材/A7S3#1/Card 1/220808_GWR_1_0001.MP4': 'A7S3#1',
 '/Volumes/thom_7_for_Mac/tmp/素材/A7S3#1/Card 1/220808_GWR_1_0002.MP4': 'A7S3#1',
 '/Volumes/thom_7_for_Mac/tmp/素材/A7S3#1/Card 1/220808_GWR_1_0003.MP4': 'A7S3#1',
 '/Volumes/thom_7_for_Mac/tmp/素材/A7S3#1/Card 1/220808_GWR_1_0004.MP4': 'A7S3#1',
 '/Volumes/thom_7_for_Mac/tmp/素材/A7S3#1/Card 1/220808_GWR_1_0005.MP4': 'A7S3#1',
 '/Volumes/thom_7_for_Mac/tmp/素材/A7S3#1/Card 1/220808_GWR_1_0006.MP4': 'A7S3#1',
 '/Volumes/thom_7_for_Mac/tmp/素材/A7S3#1/Card 2/220808_GWR_1_0026.MP4': 'A7S3#1',
 '/Volumes/thom_7_for_Mac/tmp/素材/A7S3#1/Card 2/220808_GWR_1_0027.MP4': 'A7S3#1',
 '/Volumes/thom_7_for_Mac/tmp/素材/A7S3#1/Card 2/220808_GWR_1_0028.MP4': 'A7S3#1',
 '/Volumes/thom_7_for_Mac/tmp/素材/A7S3#1/Card 2/220808_GWR_1_0029.MP4': 'A7S3#1',
 '/Volumes/thom_7_for_Mac/tmp/素材/A7S3#1/Card 2/220808_GWR_1_0030.MP4': 'A7S3#1',
 '/Volumes/thom_7_for_Mac/tmp/素材/A7S3#1/Card 2/220808_GWR_1_0031.MP4': 'A7S3#1',
 '/Volumes/thom_7_for_Mac/tm